In [2]:
import re
import jieba
from gensim.models import word2vec
from nltk.classify import NaiveBayesClassifier
from sklearn import naive_bayes
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))
#把停用词转换为一个列表存储
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list
df = pd.read_csv('test.csv')
X = df[['text']]
y = df.type
X['cut_text'] = X.text.apply(chinese_word_cut)
print(X.cut_text[:5])
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.01, random_state=1)
stop_words_file = 'F:\python\数据分析结课实验\data/停用词.txt'
stopwords = get_custom_stopwords(stop_words_file)
# 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
max_df = 0.8
# 在低于这一数量的文档中出现的关键词（过于独特），去除掉。
min_df = 4 
vect = CountVectorizer(max_df = max_df,min_df = min_df,token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',stop_words=frozenset(stopwords))
term_matrix = pd.DataFrame(vect.fit_transform(X_train.cut_text[0:300]).toarray(), columns=vect.get_feature_names())

0    啊呀呀 ！ 要死 啦 ！ 么 么 么 ！ 只 穿 外套 就 好 了 ， 我 认为 里面 那 ...
1                         风格 不 一样 嘛 ， 都 喜欢 ！ 最 喜欢 哪张 ？
2    好 呀 ， 试试 D . I . Y . 去死皮 面膜 1 . 将 燕麦片 加水 中 浸泡 ...
3    张 1 老师 ， 谢谢 侬 的 1 信任 ！ 粉丝 多少 无所谓 重在 质地 近日 发现 一...
4    第二 條看 來 有點 吸引力 呵呵 【 美国 相亲 节目 与 中国 的 1 几大 不同 】 ...
Name: cut_text, dtype: object


D:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['若果'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [81]:
import numpy as np
num_lists=[i for i in range(0,10000,500)]
# num_lists.append(len(X_train.cut_text))

frames=[]
for i in range(len(num_lists)-1):
    j=i+1
    print(num_lists[i],num_lists[j])
    term_matrix = pd.DataFrame(vect.fit_transform(X_train.cut_text[num_lists[i]:num_lists[j]]).toarray(), columns=vect.get_feature_names())
    frames.append(term_matrix)
result=pd.concat(frames) 
print(result.shape)
# term_matrix1 = pd.DataFrame(vect.fit_transform(X_train.cut_text[0:200]).toarray(), columns=vect.get_feature_names())
# term_matrix2 = pd.DataFrame(vect.fit_transform(X_train.cut_text[200:400]).toarray(), columns=vect.get_feature_names())
# term_matrix.
# term_matrix.append(term_matrix2)
# print(len(X_train.cut_text))
# print(term_matrix1.shape)

0 500
500 1000
1000 1500
1500 2000
2000 2500
2500 3000
3000 3500
3500 4000
4000 4500
4500 5000
5000 5500
5500 6000
6000 6500
6500 7000
7000 7500
7500 8000
8000 8500
8500 9000
9000 9500


D:\anaconda\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


(9500, 42582)


In [3]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
nb = MultinomialNB()
pipe = make_pipeline(vect, nb)
pipe.steps

[('countvectorizer',
  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=0.8, max_features=None, min_df=4,
          ngram_range=(1, 1), preprocessor=None,
          stop_words=frozenset({'便于', '设或', '哗', '用', '”', '甚且', '临', '乘', '然则', '啐', '纵', '说来', '非但', '以及', '呀', '在', '为止', '这般', '那么些', '一旦', '不光', '咦', '哉', '哦', '你', '假如', '哟', '对', '不成', '总之', '据', '全体', '?', '固然', '哼', '而外', '对于', '别是', '、', '总的说来', '就算', '此时', '始而', '以故', '只是', '紧接着', '处在', '且', '非独', ...是说', '云云', '谁知', '咚', '哪年', '要不是', '再其次', '万一', '各位', '这儿', '接着', '那般', '别说', '多么', '此地', '诸', '都'}),
          strip_accents=None, token_pattern='(?u)\\b[^\\d\\W]\\w+\\b',
          tokenizer=None, vocabulary=None)),
 ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]

In [7]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train.cut_text, y_train, cv=5, scoring='accuracy').mean()

D:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['若果'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
D:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['若果'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
D:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['若果'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
D:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['若果'] not in stop_words.
  'stop_words.' % sorted(inc

0.6198575913510876

In [8]:
pipe.fit(X_train.cut_text, y_train)

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=None, min_df=4,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=...e, vocabulary=None)), ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [40]:
pipe.predict(X_test.cut_text)[0:200]

array([2, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 1, 0, 0, 0, 0, 0,
       2, 3, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 2, 2, 0, 0, 2, 0, 0, 0,
       0, 0, 2, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 2,
       0, 1, 3, 0, 0, 0, 3, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 2,
       1, 1, 0, 1, 0, 2, 0, 0, 0, 1, 0, 0, 2, 2, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 3, 0, 1, 0, 0, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int64)

In [11]:
y_pred = pipe.predict(X_test.cut_text)

In [12]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred)

0.6219054763690923

In [13]:
metrics.confusion_matrix(y_test, y_pred)

array([[1341,  112,  109,   35],
       [ 236,  123,   45,   18],
       [ 203,   59,  169,   16],
       [ 130,   23,   22,   25]], dtype=int64)

In [33]:
from snownlp import SnowNLP
from interval import Interval
def get_sentiment(text):
    return SnowNLP(text).sentiments
y_pred_snownlp = X_test.cut_text.apply(get_sentiment)
y_pred_snownlp[0:5]

9922      0.997301
205908    0.303299
164063    0.947852
28882     0.999999
202471    0.999923
Name: cut_text, dtype: float64

SyntaxError: invalid syntax (<ipython-input-23-2498d2cefbbd>, line 2)

In [48]:
def dd(x):
    if x in Interval(0,0.1):
        return 3
    if x in Interval(0.1, 0.3, lower_closed=False):
        return 2
    if x in Interval(0.3, 0.5, lower_closed=False):
        return 1
    else:
        return 0
    
    
y_pred_snownlp_normalized = y_pred_snownlp.apply(dd)
y_pred_snownlp[0:5]
y_pred_snownlp_normalized[:5]

9922      0
205908    1
164063    0
28882     0
202471    0
Name: cut_text, dtype: int64

In [51]:
metrics.accuracy_score(y_test, y_pred_snownlp_normalized)
metrics.confusion_matrix(y_test, y_pred_snownlp_normalized)

array([[1080,  118,  138,  261],
       [ 222,   30,   40,  130],
       [ 246,   36,   51,  114],
       [ 107,   13,   21,   59]], dtype=int64)

In [50]:
y_pred_snownlp[0:5]

9922      0.997301
205908    0.303299
164063    0.947852
28882     0.999999
202471    0.999923
Name: cut_text, dtype: float64